In [23]:
import pandas as pd
import re

def parseFile(path):
    """
    
    """
    top_lines = [re.compile("top +- +(.*) +up +(.*) +day, + (.*), + (.*) +user, +load +average: +(.*), +(.*), +(.*)$"),
                 re.compile("Tasks: +(.*) +total, +(.*) +running, +(.*) +sleeping, +(.*) +stopped, +(.*) +zombie$"),
                 re.compile("%Cpu\(s\): +(.*) +us, +(.*) +sy, +(.*) +ni, +(.*) +id, +(.*) +wa, +(.*) +hi, +(.*) +si, +(.*) +st$"),
                 re.compile("KiB +Mem: +(.*) +total, +(.*) +used, +(.*) +free, +(.*) +buffers$"),
                 re.compile("KiB +Swap: +(.*) +total, +(.*) +used, +(.*) +free. +(.*) +cached +Mem$"),
                 re.compile("([0-9]+) +([^ ]+) +([^ ]+) +([\-0-9]+) +([0-9]+) +([0-9]+) +([0-9]+) +([^ ]+) +([0-9]*.[0-9]*) +([0-9]*.[0-9]*) +([0-9]*:[0-9]*.[0-9]*) +(.*)$")
                ]
    
    MAX_LINES = 20000
    tdata = []
    pdata = []
    missed = []
    
    with open(path, 'r') as file:
        i = -1
        line = file.readline()
        isTop = True    
        while line:
            if(top_lines[0].search(line)):
                isTop = True
                i += 1
            if isTop:
                entry = {}
                if(top_lines[0].search(line)):
                    #print(top_lines[0].search(line))
                    entry = {'timestamp': top_lines[0].search(line).group(1),
                             'upday': top_lines[0].search(line).group(2),
                             'uptime': top_lines[0].search(line).group(3),
                             'users': top_lines[0].search(line).group(4),
                             'load average': [top_lines[0].search(line).group(5),
                                              top_lines[0].search(line).group(6), 
                                              top_lines[0].search(line).group(7)]
                            }
                line = file.readline()
                if(top_lines[1].search(line)):
                    #print(top_lines[1].search(line))
                    entry.update({'total tasks': top_lines[1].search(line).group(1),
                                  'running tasks': top_lines[1].search(line).group(2),
                                  'sleeping tasks': top_lines[1].search(line).group(3),
                                  'stopped tasks': top_lines[1].search(line).group(4),
                                  'zombie tasks': top_lines[1].search(line).group(5),
                                 })
                line = file.readline()
                if(top_lines[2].search(line)):
                    #print(top_lines[2].search(line))
                    entry.update({'us': top_lines[2].search(line).group(1),
                                  'sy': top_lines[2].search(line).group(2),
                                  'ni': top_lines[2].search(line).group(3),
                                  'id': top_lines[2].search(line).group(4),
                                  'wa': top_lines[2].search(line).group(5),
                                  'hi': top_lines[2].search(line).group(6),
                                  'si': top_lines[2].search(line).group(7),
                                  'st': top_lines[2].search(line).group(8)
                                 })
                line = file.readline()
                if(top_lines[3].search(line)):
                    #print(top_lines[3].search(line))
                    entry.update({'total mem': top_lines[3].search(line).group(1),
                                 'used mem': top_lines[3].search(line).group(2),
                                 'free mem': top_lines[3].search(line).group(3),
                                 'disk buffer/cache': top_lines[3].search(line).group(4)       
                                 })
                line = file.readline()
                if(top_lines[4].search(line)):
                    #print(top_lines[4].search(line))
                    entry.update({'total swap': top_lines[4].search(line).group(1),
                                  'used swap': top_lines[4].search(line).group(2),
                                  'free swap': top_lines[4].search(line).group(3),
                                  'avail mem': top_lines[4].search(line).group(4)
                                 })
                #print(entry)
                tdata.append(entry)
                line = file.readline()
                line = file.readline() #skip blank line
                line = file.readline() #skip column header line
                isTop = False
                
            else:
                if(top_lines[5].search(line)):
                    pentry = {'index': i,
                             'PID': top_lines[5].search(line).group(1),
                             'USER': top_lines[5].search(line).group(2),
                             'PR': top_lines[5].search(line).group(3),
                             'NI': top_lines[5].search(line).group(4),
                             'VIRT': top_lines[5].search(line).group(5),
                             'RES': top_lines[5].search(line).group(6),
                             'SHR': top_lines[5].search(line).group(7),
                             'S': top_lines[5].search(line).group(8),
                             '%CPU': top_lines[5].search(line).group(9),
                             '%MEM': top_lines[5].search(line).group(10),
                             'TIME+': top_lines[5].search(line).group(11),
                             'CMD': top_lines[5].search(line).group(12)
                             }
                   #print(pentry)
                    pdata.append(pentry)
                else:
                    print('ERROR --' + line)
                    #missed.append(line)
                line = file.readline()
            
            
            
           # if (i>MAX_LINES):
           #     return data
            
            #print(line)
            
    print(missed)        
    return tdata, pdata

parsed = parseFile('output-top.log')
topData = pd.DataFrame(parsed[0])
procData = pd.DataFrame(parsed[1])
display(topData)
display(procData)
topData.to_csv(r'topData.csv')
procData.to_csv(r'procData.csv')


[]


,avail mem,disk buffer/cache,free mem,free swap,hi,id,load average,ni,running tasks,si,...,total swap,total tasks,upday,uptime,us,used mem,used swap,users,wa,zombie tasks
0,502804,58664,279060,1046524,0.0,99.9,"[0.01, 0.03, 0.05]",0.0,1,0.0,...,1046524,110,1,3:15,0.1,738752,0,1,0.0,0
1,504440,58784,275000,1046524,0.0,99.9,"[0.04, 0.03, 0.05]",0.0,1,0.0,...,1046524,111,1,4:03,0.1,742812,0,1,0.0,0
2,504688,58792,270512,1046524,0.0,99.9,"[0.00, 0.01, 0.05]",0.0,1,0.0,...,1046524,112,1,4:05,0.1,747300,0,1,0.0,0
3,504696,58792,270508,1046524,0.0,99.9,"[0.00, 0.01, 0.05]",0.0,1,0.0,...,1046524,112,1,4:05,0.1,747304,0,1,0.0,0
4,504704,58792,270624,1046524,0.0,99.9,"[0.00, 0.01, 0.05]",0.0,1,0.0,...,1046524,112,1,4:06,0.1,747188,0,1,0.0,0
5,504716,58792,270500,1046524,0.0,99.9,"[0.00, 0.01, 0.05]",0.0,1,0.0,...,1046524,112,1,4:06,0.1,747312,0,1,0.0,0
6,504724,58792,270624,1046524,0.0,99.9,"[0.00, 0.01, 0.05]",0.0,1,0.0,...,1046524,112,1,4:06,0.1,747188,0,1,0.0,0
7,504732,58800,270500,1046524,0.0,99.9,"[0.00, 0.01, 0.05]",0.0,3,0.0,...,1046524,112,1,4:06,0.1,747312,0,1,0.0,0
8,504740,58800,270628,1046524,0.0,99.9,"[0.00, 0.01, 0.05]",0.0,2,0.0,...,1046524,112,1,4:06,0.1,747184,0,1,0.0,0
9,504748,58800,270504,1046524,0.0,99.9,"[0.00, 0.01, 0.05]",0.0,1,0.0,...,1046524,112,1,4:06,0.1,747308,0,1,0.0,0


,%CPU,%MEM,CMD,NI,PID,PR,RES,S,SHR,TIME+,USER,VIRT,index
0,6.6,5.5,Xorg,0,995,20,55672,S,12376,0:17.41,root,240352,0
1,6.6,1.5,lxterminal,0,1659,20,15116,S,10480,0:03.21,remnux,507716,0
2,0.0,0.3,init,0,1,20,2784,S,1456,0:00.60,root,33500,0
3,0.0,0.0,kthreadd,0,2,20,0,S,0,0:00.00,root,0,0
4,0.0,0.0,ksoftirqd/0,0,3,20,0,S,0,0:00.04,root,0,0
5,0.0,0.0,kworker/0:0,0,4,20,0,S,0,0:00.00,root,0,0
6,0.0,0.0,kworker/0:0H,-20,5,0,0,S,0,0:00.00,root,0,0
7,0.0,0.0,rcu_sched,0,7,20,0,S,0,0:00.95,root,0,0
8,0.0,0.0,rcuos/0,0,8,20,0,S,0,0:01.47,root,0,0
9,0.0,0.0,rcu_bh,0,9,20,0,S,0,0:00.00,root,0,0


In [8]:
entry = {'test1': 'h',
        'test2': 'asbds'}
print(entry)
entry.update({'tester3': 123, 'tester66': 542, 'test2': 'hello'})
print(entry)


{'test1': 'h', 'test2': 'asbds'}
{'test1': 'h', 'test2': 'hello', 'tester3': 123, 'tester66': 542}


In [22]:
topData['timestamp']

0       12:49:55
1       13:38:06
2       13:40:15
3       13:40:16
4       13:40:18
5       13:40:19
6       13:40:20
7       13:40:21
8       13:40:22
9       13:40:23
10      13:40:24
11      13:40:26
12      13:40:27
13      13:40:28
14      13:40:29
15      13:40:30
16      13:40:31
17      13:40:33
18      13:40:34
19      13:40:35
20      13:40:36
21      13:40:37
22      13:40:38
23      13:40:39
24      13:40:41
25      13:40:42
26      13:40:43
27      13:40:44
28      13:40:45
29      13:40:46
          ...   
3410    14:45:59
3411    14:46:00
3412    14:46:01
3413    14:46:02
3414    14:46:03
3415    14:46:04
3416    14:46:05
3417    14:46:07
3418    14:46:08
3419    14:46:09
3420    14:46:10
3421    14:46:11
3422    14:46:12
3423    14:46:14
3424    14:46:15
3425    14:46:16
3426    14:46:17
3427    14:46:18
3428    14:46:19
3429    14:46:21
3430    14:46:22
3431    14:46:23
3432    14:46:24
3433    14:46:25
3434    14:46:26
3435    14:46:27
3436    14:46:29
3437    14:46: